In [48]:
from strategie import strategie,teststrategie
import sqlite3
import itertools
import pickle
import pandas as pd
import matplotlib.pyplot as plt

In [49]:
parametre={
    'periode_analyse_cours_action':"Month",
    'centile_max_action' : 0.4,
    'type_moyenne_glissante_cac':"Trimester",
    'centile_max_cac' : 0.5,
    'taux_gain':0.2,
    'periode_attente_vente':60,
}

name="Total"
print(teststrategie(name,parametre))

{'Ratio Moyen du gain ': 0.08945172216829395, 'Precision': 0.475, 'nb_strategie_excute': 80}


In [50]:
grid_params={
    'periode_analyse_cours_action':["Trimester","Year","Month"],
    'centile_max_action' : [0.5,0.4,0.3],
    'type_moyenne_glissante_cac':["Month","Trimester","Week"],
    'centile_max_cac' : [0.5,0.4,0.3],
    'taux_gain':[0.05,0.1,0.15,0.2],
    'periode_attente_vente':[90,60,30,20]
}


In [51]:
simulations_to_run = [dict(zip(grid_params.keys(), _)) for _ in itertools.product(*grid_params.values())]
simulations_to_run = [frozenset(simulation.items()) for simulation in simulations_to_run]
len(simulations_to_run)


1296

In [52]:
try:
    with open('grid_search_'+name+'.pickle', 'rb') as fd:
        simulation_scores = pickle.load(fd)
except FileNotFoundError:
    simulation_scores = {}


In [ ]:
for params in simulations_to_run:
    dict_params = dict(params)
    if params in simulation_scores:
        print(f"Not running {dict_params}")
    else:
        print(f"Running {dict_params}")
        scores = teststrategie(name,dict_params)
        simulation_scores[params] = scores
        with open('grid_search_'+name+'.pickle', 'wb') as fd:
            pickle.dump(simulation_scores, fd)



Running {'periode_analyse_cours_action': 'Trimester', 'centile_max_action': 0.5, 'periode_attente_vente': 90, 'centile_max_cac': 0.5, 'taux_gain': 0.05, 'type_moyenne_glissante_cac': 'Month'}
Running {'periode_analyse_cours_action': 'Trimester', 'centile_max_action': 0.5, 'centile_max_cac': 0.5, 'taux_gain': 0.05, 'periode_attente_vente': 60, 'type_moyenne_glissante_cac': 'Month'}
Running {'periode_analyse_cours_action': 'Trimester', 'centile_max_action': 0.5, 'centile_max_cac': 0.5, 'taux_gain': 0.05, 'periode_attente_vente': 30, 'type_moyenne_glissante_cac': 'Month'}
Running {'periode_analyse_cours_action': 'Trimester', 'centile_max_action': 0.5, 'periode_attente_vente': 20, 'centile_max_cac': 0.5, 'taux_gain': 0.05, 'type_moyenne_glissante_cac': 'Month'}
Running {'periode_analyse_cours_action': 'Trimester', 'centile_max_action': 0.5, 'periode_attente_vente': 90, 'taux_gain': 0.1, 'centile_max_cac': 0.5, 'type_moyenne_glissante_cac': 'Month'}
Running {'periode_analyse_cours_action': 

In [ ]:
liste=[]
for k, v in simulation_scores.items() :
    print(v)

In [ ]:

scores_df = pd.DataFrame(dict(k) | v for k, v in simulation_scores.items() if k in simulations_to_run and type(v)==dict )
scores_df.rename(columns = {'Ratio Moyen du gain ':'MeanGain'}, inplace = True)
scores_df["risque"]=1-scores_df["Precision"]


In [ ]:
scores_df.pipe(lambda x: x[x.Precision == x.Precision.max()]  )

In [ ]:
scores_df.pipe(lambda x: x[x.taux_gain==0.05 ]  )

In [ ]:
scores_df.pipe(lambda x: x[x.MeanGain == x.MeanGain.max() ])


In [ ]:
scores_df.pipe(lambda x: x[x.Precision > 0.5]).plot.scatter('Precision', 'MeanGain', figsize=(10, 10), s=70, alpha=0.5)
plt.xlim([0.3, 1.1])
plt.ylim([0, 0.3])
plt.savefig('gridsearch_gainprecision'+name+'.png')

Risque LVMH

In [ ]:
name="LVMH"
try:
    with open('nuage_point_nbstrat_' + name + '.pickle', 'rb') as fd:
        simulation_scores = pickle.load(fd)
except FileNotFoundError:
    simulation_scores = {}

for params in simulations_to_run:
    dict_params = dict(params)
    if params in simulation_scores:
        print(f"Not running {dict_params}")
    else:
        print(f"Running {dict_params}")
        scores = teststrategie(name, dict_params)
        simulation_scores[params] = scores
        with open('nuage_point_nbstrat_' + name + '.pickle', 'wb') as fd:
            pickle.dump(simulation_scores, fd)

In [ ]:
scores_df = pd.DataFrame(dict(k) | v for k, v in simulation_scores.items() if k in simulations_to_run and type(v)==dict )
scores_df.rename(columns = {'Ratio Moyen du gain ':'MeanGain'}, inplace = True)
scores_df["risque"]=1-scores_df["Precision"]
print(scores_df)


In [ ]:
scores_df["c1"] = (scores_df["nb_strategie_excute"] / 103) * (
            scores_df["nb_strategie_excute"] / 103) - (scores_df["risque"]) * (scores_df["risque"])

scores_df.pipe(lambda x: x[x.taux_gain==0.2 ]).pipe(lambda x: x[x.risque<0.5 ]).pipe(lambda x: x[x.MeanGain>0.10 ])


In [ ]:

scores_df.pipe(lambda x: x[ x.taux_gain==0.2]).plot.scatter('risque', 'nb_strategie_excute', figsize=(10, 10), s=70, alpha=0.5)
"""plt.xlim([0, 1.1])
plt.ylim([0, 0.3])"""
plt.savefig('precision_nbstart'+name+'.png')

Risque Total

In [ ]:
name="Total"
try:
    with open('nuage_point_nbstrat_' + name + '.pickle', 'rb') as fd:
        simulation_scores_total = pickle.load(fd)
except FileNotFoundError:
    simulation_scores_total = {}

for params in simulations_to_run:
    dict_params = dict(params)
    if params in simulation_scores_total:
        print(f"Not running {dict_params}")
    else:
        print(f"Running {dict_params}")
        scores = teststrategie(name, dict_params)
        simulation_scores_total[params] = scores
        with open('nuage_point_nbstrat_' + name + '.pickle', 'wb') as fd:
            pickle.dump(simulation_scores_total, fd)

In [ ]:
scores_df_total = pd.DataFrame(dict(k) | v for k, v in simulation_scores_total.items() if k in simulations_to_run and type(v)==dict )
scores_df_total.rename(columns = {'Ratio Moyen du gain ':'MeanGain'}, inplace = True)
scores_df_total["risque"]=1-scores_df_total["Precision"]
scores_df_total["c1"]=-scores_df_total["risque"]**2+scores_df_total["nb_strategie_excute"]/200
print(scores_df_total)

In [ ]:
scores_df_total.pipe(lambda x: x[x.Precision == x.Precision.max()])

In [ ]:
scores_df_total.pipe(lambda x: x[ x.taux_gain==0.2]).plot.scatter('risque', 'nb_strategie_excute', figsize=(10, 10), s=70, alpha=0.5)
plt.savefig('precision_nbstart'+name+'.png')

In [ ]:
scores_df_total["c1"]=(scores_df_total["nb_strategie_excute"]/121)*(scores_df_total["nb_strategie_excute"]/121)-(scores_df_total["risque"])*(scores_df_total["risque"])

In [ ]:
scores_df_total.pipe(lambda x: x[x.taux_gain==0.2 ]).pipe(lambda x: x[x.risque<0.5 ]).pipe(lambda x: x[x.MeanGain>0.15 ])